In [33]:
# all paths
patientsRecordsPath = "wasb:///hive_patient_master_dd-1lab-20171016-140000.csv"
fakeNamesPath = "wasb:///demographics/fake-names.txt"
demographicsPath = "wasb:///demographics/all-patients.txt"
patientData = sc.textFile(patientsRecordsPath)
fakeNames = sc.textFile(fakeNamesPath)
patientHeader = patientData.first()
patientDataWoHeader = patientData.filter(lambda x: x !=patientHeader)

In [2]:
import csv
from io import StringIO
from collections import namedtuple

In [3]:
patientFields   = patientHeader.replace(" ","_").replace("/","_").split("\t")

In [4]:
Patient   = namedtuple('Patient', patientFields, verbose=False)

In [5]:
def parsePatient(row):
    reader = csv.reader(StringIO(row), delimiter='\t')
    row=next(reader)
    return Patient(*row)

In [6]:
patients = patientDataWoHeader.map(parsePatient)

In [30]:
patientZip = patients.zipWithIndex()
nameZip = fakeNames.zipWithIndex()
patientMap = patientZip.map(lambda x : (x[1], x[0]))
nameMap = nameZip.map(lambda x : (x[1], x[0]))
joinResult = patientMap.join(nameMap).map(lambda x: x[1])


[(Patient(pat_master_id='55677364', age_in_years='25', pm_gender='F', pm_state='TX', three_digit_zip='777', pm_country='', pm_kba_auth_flag='Y', pm_marital_status='', pm_ethnic_grp='', opt_in_flag='Y', opt_in_timestamp='2015-08-04 15:08:46', opt_out_timestamp='', first_opt_in_timestamp='2015-08-04 15:08:46', prs_update_date='2015-08-05 01:08:44'), 'Yandel Davis'), (Patient(pat_master_id='11046338', age_in_years='76', pm_gender='M', pm_state='LA', three_digit_zip='701', pm_country='', pm_kba_auth_flag='Y', pm_marital_status='', pm_ethnic_grp='', opt_in_flag='Y', opt_in_timestamp='2016-02-10 10:02:15', opt_out_timestamp='', first_opt_in_timestamp='2016-02-10 10:02:15', prs_update_date='2016-02-11 03:02:35'), 'Reed Lucas')]

In [42]:
allPatients = joinResult.map(lambda x: "{}|{}|{}|{}".format(x[0].pat_master_id, x[1], x[0].pm_gender,x[0].three_digit_zip))

In [43]:
allPatients.saveAsTextFile(demographicsPath)